# Downloading genbank accessions from .xlsx table

In [10]:
import pandas as pd
from Bio import Entrez
import os

In [4]:
spp_accession = pd.read_excel("spp_accession.xlsx")

In [5]:
spp_accession

,Species,18S,28S,abdA,EF1,EF2,LWR,WNT
0,Formica_fusca,FJ824197.1,FJ824242.1,FJ824331.1,AY334415.1,EF013263.1,KJ920611.1,LT977286.1
1,Linepithema_humile,AH012894.2,AH012910.2,AY525327.1,EF013277.2,EF013277.2,FJ161830.1,AY233596.1
2,Pseudomyrmex_gracilis,AH012887.2,AH012903.2,DQ352945.1,NaN,NaN,KR829388.1,KR829015.1
3,Solenopsis_invicta,AY334566.1,AY325955.1,DQ352941.1,KP730149.1,EF013348.1,EF013642.1,KP730228.1


In [11]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

In [14]:
create_dir("seed_fasta/") # Creating dir that will contain fasta_files

In [17]:
#os.chdir("..") 
os.getcwd()

'/home/gabriel/Dropbox/repos/genbank_phylogeny'

In [26]:
def download_acc_from_dataframe(dataframe):
    for (column_name, column) in dataframe.iteritems():
        print(column_name, column)

In [27]:
download_acc_from_dataframe(spp_accession)

Species 0            Formica_fusca
1       Linepithema_humile
2    Pseudomyrmex_gracilis
3       Solenopsis_invicta
Name: Species, dtype: object
18S 0    FJ824197.1
1    AH012894.2
2    AH012887.2
3    AY334566.1
Name: 18S, dtype: object
28S 0    FJ824242.1
1    AH012910.2
2    AH012903.2
3    AY325955.1
Name: 28S, dtype: object
abdA 0    FJ824331.1
1    AY525327.1
2    DQ352945.1
3    DQ352941.1
Name: abdA, dtype: object
EF1 0    AY334415.1
1    EF013277.2
2           NaN
3    KP730149.1
Name: EF1, dtype: object
EF2 0    EF013263.1
1    EF013277.2
2           NaN
3    EF013348.1
Name: EF2, dtype: object
LWR 0    KJ920611.1
1    FJ161830.1
2    KR829388.1
3    EF013642.1
Name: LWR, dtype: object
WNT 0     LT977286.1
1     AY233596.1
2     KR829015.1
3     KP730228.1
Name: WNT, dtype: object


In [25]:
for gene in spp_accession.columns[1:]:
    path = "seed_fasta/{}/".format(gene)
    create_dir(path)

In [8]:
handle = Entrez.efetch(db='protein', id="YP_009705869", rettype='fasta', retmode='text')
handle.read()

'>YP_009705869.1 cytochrome c oxidase subunit III (mitochondrion) [Linepithema humile]\nMSVTHNHPFHLVSVSPWPIIISFNLMNNLIMVISWFHKMNYFTLLTIPGTLLCMYQWWRDVTREATLQGC\nHTPLVYNGMRMGMILFIVSEILFFFPFFVAYFHSSLSPAMEIGQSWPPLGIIPFNPFDIPLLNTIILVSS\nGLTVTWSHHAMLNNNYKESTISLLMTILLGIYFSALQLIEYIEAPFTISDSVYGSSFFVATGFHGLHVII\nGTMFLFFTFYRMNLLHFSSIHHFGFEAAAWYWHFVDVVWLFLYVSIYWWSF\n\n'